In [1]:
import cv2 as cv
import numpy as np
import HandTrackingModule as htm
import math
import subprocess

In [ ]:
def set_volume(volume_level):
    # Create an AppleScript command to set the volume
    script = f"set volume output volume {volume_level}"
    # Execute the AppleScript command
    subprocess.run(["osascript", "-e", script])

# Function to get the current system volume level
def get_volume():
    # Create an AppleScript command to get the current volume level
    script = "output volume of (get volume settings)"
    # Run the command and capture the output
    result = subprocess.run(["osascript", "-e", script], capture_output=True, text=True)
    # Return the output volume as an integer
    return int(result.stdout.strip())

# Set the initial volume to 30%
set_volume(30)  
# Get the current volume level
current_volume = get_volume()
# Print the current volume level
print(f"Current volume level: {current_volume}")


In [ ]:
# Another print statement to show the volume level
print(get_volume())

In [ ]:
# Initialize the webcam
webcam = cv.VideoCapture(0)
# Set the width and height of the webcam video feed
wCam, hCam = 640, 480
webcam.set(3, wCam)  # Set width
webcam.set(4, hCam)  # Set height

# Initialize the hand detector with a detection confidence threshold
detector = htm.handdetectors(detectionCon=0.7)

# Initialize the volume variable
vol = 0

# Start an infinite loop to process each frame from the webcam
while True:
    # Capture a frame from the webcam
    ok, frame = webcam.read()

    # Uncomment this line if you want to flip the frame horizontally
    # frame = cv.flip(frame, 1)

    # If the frame is not read properly, exit the loop
    if not ok:
        break

    # Start a TickMeter to calculate FPS
    tm = cv.TickMeter()
    tm.start()

    # Detect hands in the frame
    hand_detected = detector.findhands(frame)
    # Get the positions of hand landmarks
    lm_list = detector.findposition(frame, False)

    # If landmarks are detected
    if len(lm_list) != 0:
        # Get the coordinates of the thumb tip (landmark 4)
        x1, y1 = lm_list[4][1], lm_list[4][2]
        # Get the coordinates of the index finger tip (landmark 8)
        x2, y2 = lm_list[8][1], lm_list[8][2]
        # Calculate the midpoint between the thumb and index finger
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        # Draw circles on the thumb, index finger, and midpoint
        cv.circle(hand_detected, (x1, y1), 10, (255, 0, 255), -1)
        cv.circle(hand_detected, (x2, y2), 10, (255, 0, 255), -1)
        cv.circle(hand_detected, (cx, cy), 10, (255, 0, 255), -1)

        # Draw a line between the thumb and index finger
        cv.line(hand_detected, (x1, y1), (x2, y2), (255, 0, 255), 3)

        # Calculate the distance between the thumb and index finger
        length = math.hypot(x2 - x1, y2 - y1)

        # Map the distance to a volume level (0 to 100)
        vol = np.interp(length, [50, 270], [0, 100])
        # Map the distance to a position on the volume bar (170 to 470 pixels)
        vol_bar = np.interp(length, [50, 270], [470, 170])

        # Set the system volume based on the hand gesture
        set_volume(vol)

        # Draw a filled rectangle representing the current volume level
        cv.rectangle(hand_detected, (30, int(vol_bar)), (70, 470), (128, 128, 128), -1)

    # Draw an empty rectangle as the volume bar background
    cv.rectangle(hand_detected, (30, 170), (70, 470), (128, 128, 128), 3)
    
    # Stop the TickMeter to calculate FPS
    tm.stop()

    # Display the FPS on the frame
    cv.putText(hand_detected, f'FPS : {str(int(round(tm.getFPS())))}', (30, 30), cv.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
    # Display the current volume level on the frame
    cv.putText(hand_detected, f'vol : {str(int(vol))}', (30, 150), cv.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)

    # Show the processed frame
    cv.imshow('frame', hand_detected)

    # Break the loop if the user presses 'q'
    if cv.waitKey(1) & 0xff == ord('q'):
        break

# Release the webcam and destroy all OpenCV windows
cv.destroyAllWindows()
webcam.release()